In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install deepface

In [ ]:
import cv2
from deepface import DeepFace
from google.colab.patches import cv2_imshow
import time
import os
from collections import Counter, defaultdict # for counting and storing confidence scores

# path to your RAVDESS video in Google Drive
video_path = "/content/drive/MyDrive/RAVDESS_Videos/Actor_02/02-01-05-02-02-01-02.mp4"

# open the video file using opencv
cap = cv2.VideoCapture(video_path)

# check to see if OpenCV opened the video successfully
if not cap.isOpened():
    print("Error: The video file cannot be opened...")
else:
    print("Processing video...")

frame_count = 0
total_start_time = time.time()  # start tracking total video processing time
detected_emotions = [] # store detected emotions in an array
confidence_scores = defaultdict(float) # store confidence scores ( how confident is the detection for each emotion ) 

# process the video frame by frame
while cap.isOpened():
    frame_start_time = time.time() # track time for specific frame
    ret, frame = cap.read() # ret is a boolean check if frame is being read successfully

    if not ret:
       break  # exit when video ends

    frame_count += 1 # count processed frames#

    # Skipping processing unless its every 5th frame. 
    if frame_count % 5 != 0:
      continue

    try: 
        result = DeepFace.analyze(
            img_path=frame,
            actions=["emotion"],
            detector_backend="retinaface", # deepface emotion detection model options:(mtcnn, opencv & dlib)
            enforce_detection=False # returns 'neutral' if no face detected rather than error if no face found.
        )


       # get a dominant emotion and its confidence score
        dominant_emotion = result[0]['dominant_emotion']  # Most confident emotion
        confidence_score = result[0]['emotion'][dominant_emotion]  # Confidence percentage

        detected_emotions.append(dominant_emotion)  # Store detected emotion
        confidence_scores[dominant_emotion] += confidence_score  # Accumulate confidence

        print(f"🧠 Frame {frame_count}: Detected Emotion - {dominant_emotion} ({confidence_score:.2f}%)")  # Log detected emotion

        # Draw emotion text on frame
        cv2.putText(frame, f"Emotion: {dominant_emotion}", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # show frame (displays analysis frame by frame)
        cv2_imshow(frame)

    except Exception as e:
        print(f"⚠️ Error processing frame {frame_count}:", e)

    # measure processing time for this frame
    frame_end_time = time.time()
    frame_processing_time = frame_end_time - frame_start_time
    print(f"⏳ Time taken for frame {frame_count}: {frame_processing_time:.4f} seconds")

# releasing resources
cap.release()

# calculate & print performance stats
total_end_time = time.time()
total_processing_time = total_end_time - total_start_time
print(f"✅ Total Video Processing Time: {total_processing_time:.2f} seconds")
print(f"⚡ Average FPS: {frame_count / total_processing_time:.2f}")

# determine the most common detected emotion
if detected_emotions:
    most_common_emotion = Counter(detected_emotions).most_common(1)[0][0]
    print(f"🎭 Most Common Emotion Detected (Occurrence-Based): {most_common_emotion}")

    # determine the emotion with highest confidence
    highest_confidence_emotion = max(confidence_scores, key=confidence_scores.get)
    print(f"🔝 Most Confident Emotion Detected (Confidence-Based): {highest_confidence_emotion}")

else:
    print("❌ No valid emotions detected.")